# Model Processing

## Imports & General Settings 

In [90]:
import unittest
import os
import sys
import time
import pathlib

import matplotlib as plt
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import trange, tqdm

# Our imports
from data import WaveletTransform, AFECGDataset
import dsp
from model.blocks import ConvNet, BRNN, SoftmaxAttention
from model.baseline import Baseline
from training import train


%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [91]:
test = unittest.TestCase()
plt.rcParams.update({'font.size': 12})
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


## Dataset creation

In [6]:
dataset_name = 'afdb'
dataset = AFECGDataset(dataset_name, '../data/files/')

In [7]:
dataset.load()

25it [00:00, 249.40it/s]

Preparing 1397 samples


1397it [00:04, 336.28it/s]


Elapsed time: 4165.770053863525 ms
Skipped 1397 files which had a backup


In [8]:
data, label = dataset[0]
data.shape

torch.Size([20, 20, 375])

In [9]:
images_per_sample = 20
total_data_size = len(dataset)
print("Total data size: ", total_data_size)

Total data size:  1397


In [10]:
# data = [dataset[i][0] for i in range(total_data_size)]
# labels = [dataset[i][1] for i in range(total_data_size)]

### Example of one ECG sample

In [11]:
# samples, label = data[0], labels[0]
# print('P-signal: ', samples)
# print('Has AF: ', 'Yes' if label == 1 else 'No')

In [12]:
# to_wavelet = WaveletTransform(wavelet.Morlet(6), resample=20)
# t = to_wavelet(data[0][0])
# image_test = (t * 100 * 255).int() # Simple visualization test
# transforms.ToPILImage()(image_test).show()

##  Wavelet Transform

In [13]:
# Total data size is 1397
# You can choose the data size 
data_size = len(dataset)

In [16]:
# x0 = x_train[0][0].float()
# encoder_cnn = ConvNet((375, 20))

# display(x0.unsqueeze(0).shape)
# h = encoder_cnn(x0.unsqueeze(0))
# print(h.shape)

# test.assertEqual(h.dim(), 2)
# test.assertSequenceEqual(h.shape, (1, 50))

## BRNN

In [17]:
display(BRNN(50, 50, images_per_sample))

BRNN(
  (bi_rnn): RNN(50, 50, num_layers=20, bidirectional=True)
)

## Attention

Notations:

* $Y = \left[ y_1, \ldots, y_T \right]$ – the input matrix of size $\left( N \times T \right)$, where $N$ is the number of features in a single output vector of the BRNN

* $w_\mathrm{att}$ – The parameters of the attention model, of size $\left( N \times 1 \right)$, where $N$ is the number of features in a single output vector of the BRNN

* $\alpha$ – The attention weights, given as $\alpha = \mathrm{softmax} \left( w_\mathrm{att}^T Y \right)$. This is an element-wise softmax, where the output size of $\alpha$ is $\left( 1 \times T \right)$

* $h_\mathrm{att}$ – Output of the attention mechanism, given by $h_\mathrm{att} = Y \alpha^T$, of size $\left( N \times 1 \right)$, i.e. a vector of $N$ features.

In [25]:
SoftmaxAttention(100)

SoftmaxAttention()

## Baseline model

In [134]:
model = Baseline()

### Training 

In [135]:
data1, label1 = dataset[0]
data2, label2 = dataset[1]
batch_data = torch.cat([data1.unsqueeze(0) * 10, data2.unsqueeze(0) * 10], dim=0)
print(batch_data.shape)
output = model(batch_data)

torch.Size([2, 20, 20, 375])
After CNN:  torch.Size([20, 2, 50])
ATT: torch.Size([20, 2, 100])
After ATT:  torch.Size([2, 1, 100])
After ATT:  tensor([[-4.2561, -6.3109,  1.1911, -0.2955, -5.3390,  0.2070, -3.4266,  2.7582,
         -0.6609,  3.2697, -2.5230,  5.5940,  4.1184, -1.4610,  0.0082, -0.2382,
         -5.3136,  0.1817, -6.3881, -4.0600, -3.3778, -0.9883,  1.8645,  0.7423,
          2.8552, -0.5872,  0.9899,  3.7438, -3.5625, -4.0232,  0.7921,  0.8500,
          7.1969,  0.8422, -2.3092, -1.4618,  0.8833, -1.2942, -3.8356, -2.6497,
         -1.2535, -0.0089,  4.7178, -0.4363, -0.6325, -1.4977, -3.9815, -4.9241,
          2.5366, -0.8858,  4.1725, -3.7541, -2.8154, -1.4812,  3.3012,  0.6719,
         -1.6442,  1.9279, -1.7704, -3.0573, -2.9901,  1.5986, -0.8152,  4.9661,
         -1.6605,  3.9446, -0.2702,  0.3645,  0.0639, -0.0665, -1.4248, -2.3715,
         -0.6283,  4.3758, -1.1104, -1.3440, -2.5410, -1.5677, -1.1338,  2.9932,
          1.1986,  5.6360,  1.8488, -5.6310,  0

In [136]:
model = Baseline()
config = dict(
    num_workers=4,
    batch_size=90,
    learning_rate=0.001,
    weight_decay=0.01,
    num_epochs=200,
    is_notebook=True
)

train(model, dataset, config)

After CNN:  torch.Size([20, 90, 50])
ATT: torch.Size([20, 90, 100])
After ATT:  torch.Size([90, 1, 100])
After ATT:  tensor([[-0.0820, -0.0657, -0.0662,  ..., -0.0757, -0.0662,  0.0549],
        [-0.0820, -0.0657, -0.0662,  ..., -0.0757, -0.0662,  0.0549],
        [-0.0820, -0.0657, -0.0662,  ..., -0.0757, -0.0662,  0.0549],
        ...,
        [-0.0820, -0.0657, -0.0662,  ..., -0.0757, -0.0662,  0.0549],
        [-0.0820, -0.0657, -0.0662,  ..., -0.0757, -0.0662,  0.0549],
        [-0.0820, -0.0657, -0.0662,  ..., -0.0757, -0.0662,  0.0549]],
       grad_fn=<SqueezeBackward1>)
After FC:  tensor([[-0.0174,  0.1670],
        [-0.0174,  0.1670],
        [-0.0174,  0.1670],
        [-0.0174,  0.1670],
        [-0.0174,  0.1670],
        [-0.0174,  0.1670],
        [-0.0174,  0.1670],
        [-0.0174,  0.1670],
        [-0.0174,  0.1670],
        [-0.0174,  0.1670]], grad_fn=<SliceBackward>)
tensor([[0.4540, 0.5460],
        [0.4540, 0.5460],
        [0.4540, 0.5460],
        [0.4540, 0.

Traceback (most recent call last):
  File "/Users/miki/opt/miniconda3/envs/cs236781-project/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/Users/miki/opt/miniconda3/envs/cs236781-project/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/Users/miki/opt/miniconda3/envs/cs236781-project/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/Users/miki/opt/miniconda3/envs/cs236781-project/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/Users/miki/opt/miniconda3/envs/cs236781-project/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/Users/miki/opt/miniconda3/envs/cs236781-project/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
  

KeyboardInterrupt: 